In [14]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")


In [15]:
#Khởi tạo mediapie
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#Hàm tính khoảng cách

In [16]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    """
    Tính góc giữa ba điểm trong không gian 3D (đơn vị: độ)
    a, b, c là danh sách chứa tọa độ 3D: [x, y, z]
    """

    #Độ sâu ảnh
    t = 33;
    A = np.array([pointA[0], pointA[1], pointA[2]/2])  # (xA, yA, zA)
    B = np.array([pointB[0], pointB[1], pointB[2]/2 ])  # (xB, yB, zB)
    C = np.array([pointC[0], pointC[1], pointC[2]/2])  # (xC, yC, zC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle


test

In [17]:
import numpy as np

# Tọa độ có z khác 0
pointA = (1, 0, 1)
pointB = (0, 0, 0)
pointC = (0, 1, 2)

angle = calculate_angle(pointA, pointB, pointC)
print(f"Góc giữa 3 điểm là: {angle:.2f} độ") # 50.77 độ 


Góc giữa 3 điểm là: 71.57 độ


## 4. Xuống quá xâu 
Góc giữa hông, vai, đầu gối nhỏ 


In [18]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_visibility',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_visibility',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_visibility',
 'LEFT_HIP_x',
 'LEFT_HIP_y',
 'LEFT_HIP_z',
 'LEFT_HIP_visibility',
 'RIGHT_HIP_x',
 'RIGHT_HIP_y',
 'RIGHT_HIP_z',
 'RIGHT_HIP_visibility',
 'LEFT_KNEE_x',
 'LEFT_KNEE_y',
 'LEFT_KNEE_z',
 'LEFT_KNEE_visibility',
 'RIGHT_KNEE_x',
 'RIGHT_KNEE_y',
 'RIGHT_KNEE_z',
 'RIGHT_KNEE_visibility',
 'LEFT_ANKLE_x',
 'LEFT_ANKLE_y',
 'LEFT_ANKLE_z',
 'LEFT_ANKLE_visibility',
 'RIGHT_ANKLE_x',
 'RIGHT_ANKLE_y',
 'RIGHT_ANKLE_z',
 'RIGHT_ANKLE_visibility']

In [19]:
#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    # # # Lấy Z của NOSE và HIP làm chuẩn
    # nose_z = landmarks[mp_pose.PoseLandmark.NOSE.value].z
    # hip_z = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z
    # scale_factor = abs(nose_z - hip_z) if abs(nose_z - hip_z) > 0 else 1  # Tránh chia cho 0

    # # # Hàm chuẩn hóa Z theo khoảng cách mũi, hông
    # def rescale_z(z):
    #     return (z - nose_z) / (scale_factor*2)

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        # data.append([keypoint.x, keypoint.y, rescale_z(keypoint.z), keypoint.visibility])
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


Xem data chuẩn

In [20]:
#Làm sạch dữ liệu dataframe trước khi thêm dữ liệu mới
df = pd.DataFrame(columns=["video", "knee_angle", "hip_angle"])

In [21]:
#Xử lý từng video để tính khoảng cách chân, đàu gốin 
def process_frame(Video_folder, Video_name):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích frame như ảnh đầu vào
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                #Chuẩn hóa z theo nose và hip

                # Góc đầu gối trái 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # Góc đầu gối phải
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                #Láy trung bình cộng 2 góc
                knee_angle = (left_knee_angle + right_knee_angle) / 2

                #Góc hông phải
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]

                right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

                #Góc hông trái
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]

                left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)

                #Lấy trung bình cộng 2 góc
                hip_angle = (left_hip_angle + right_hip_angle) / 2

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display hip angle
                cv2.putText(image, "hip_angle", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(hip_angle, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "knee_angle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            df.loc[len(df)] = [Video_name, knee_angle, hip_angle]

            # if(knee_angle < 55):
                # plt.imshow(image)
                # plt.axis("off")  # Ẩn trục tọa độ
                # plt.show()

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_files = "20250228_150821.mp4"

# video_folder = "Data/Train/Gap_lung"
# video_files = "20250228_165326.mp4"

# video_folder = "Data/Train/Xuong_qua_xau"
# video_files = "20250228_163416.mp4"

# process_frame(video_folder, video_files)



In [22]:
import os

# Đường dẫn thư mục chứa video
video_folder = "Data/Train/Correct"
# video_folder = "Data/Train/Xuong_qua_xau"
# video_folder = "Data/Train/Chan_qua_hep"
# video_folder = "Data/Train/Gap_lung"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

df = pd.DataFrame(columns=["video", "knee_angle", "hip_angle"])

for video in video_files:
    process_frame(video_folder, video)

In [23]:
df 

,video,knee_angle,hip_angle
0,20250228_080823000_iOS (video-converter.com).mp4,129.001524,160.530631
1,20250228_080823000_iOS (video-converter.com).mp4,127.244119,159.368577
2,20250228_080823000_iOS (video-converter.com).mp4,124.274999,158.182129
3,20250228_080823000_iOS (video-converter.com).mp4,119.253241,155.348466
4,20250228_080823000_iOS (video-converter.com).mp4,115.140430,152.269858
...,...,...,...
4818,VID_20250307_091538.mp4,161.256289,169.212755
4819,VID_20250307_091538.mp4,161.468352,168.773441
4820,VID_20250307_091538.mp4,161.468435,168.274266
4821,VID_20250307_091538.mp4,160.771675,169.052345


In [24]:
#Tính toán tỉ lệ giữa vai và chân
df["ratio"] = df["knee_angle"] / df["hip_angle"]
df

,video,knee_angle,hip_angle,ratio
0,20250228_080823000_iOS (video-converter.com).mp4,129.001524,160.530631,0.803594
1,20250228_080823000_iOS (video-converter.com).mp4,127.244119,159.368577,0.798427
2,20250228_080823000_iOS (video-converter.com).mp4,124.274999,158.182129,0.785645
3,20250228_080823000_iOS (video-converter.com).mp4,119.253241,155.348466,0.767650
4,20250228_080823000_iOS (video-converter.com).mp4,115.140430,152.269858,0.756160
...,...,...,...,...
4818,VID_20250307_091538.mp4,161.256289,169.212755,0.952980
4819,VID_20250307_091538.mp4,161.468352,168.773441,0.956717
4820,VID_20250307_091538.mp4,161.468435,168.274266,0.959555
4821,VID_20250307_091538.mp4,160.771675,169.052345,0.951017


In [25]:
#Phân tích bộ data
df.describe()


,knee_angle,hip_angle,ratio
count,4823.000000,4823.000000,4823.000000
mean,110.859502,134.774010,0.802677
std,38.655077,32.007026,0.113009
min,50.971483,77.079797,0.491738
25%,75.814678,103.387857,0.708898
50%,99.647196,130.059503,0.827516
75%,154.482551,170.464389,0.902445
max,165.557473,174.829348,0.996729


# Kết luận:


## 5. Lỗi gập lưng
So Sánh tỷ lệ giữa góc hông và góc đầu gối, thường thì 2 góc sẽ có độ lớn ngang nhau